In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta, time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
!pip install -U pandas-profiling

     |████████████████████████████████| 256kB 26.1MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 10.1MB 10.7MB/s 
     |████████████████████████████████| 3.1MB 25.2MB/s 
     |████████████████████████████████| 645kB 13.4MB/s 
     |████████████████████████████████| 112kB 31.0MB/s 
     |████████████████████████████████| 1.1MB 25.7MB/s 
     |████████████████████████████████| 296kB 9.4MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27099 sha256=f84b2ad734da82b99086c06edf8462b85753eaeb7f9d5ba367abcbf03372e1a3
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107437 sha256=401970e3bc1df4f3721c2be83cafd3fce942f54665927f7f00e3689c91209673
  Stored in directory: /root/.cache/pip/wheels/c0/a3/b0/f27b1cfe32ea131a3715169132ff6d856

In [3]:
# 데이터 세트 로드

url = 'https://raw.githubusercontent.com/gentle-potato/21KDT-Project/master/DataAnalysis%26Visualization/%EA%B8%B0%ED%9B%84%20%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC%20%ED%99%9C%EC%9A%A9%ED%95%9C%20%EB%AA%A8%EA%B8%B0%20%EB%B0%9C%EC%83%9D%20%EC%98%88%EB%B3%B4/%EC%9E%91%EC%97%85/4.%20dms_seoul_avg.csv'
mosquitoDF = pd.read_csv(url, encoding='cp949')
# mosquitoDF = mosquitoDF[1:]
# mosquitoDF['date'] = pd.to_datetime(mosquitoDF['date'], format = '%Y%m%d')
mosquitoDF

,date,mosquito,temp,rain_per_day,accum_rain,wind,humidity,sunshine
0,2015-04-06,199,12.214286,5.285714,0,3.371429,68.757143,8.880000
1,2015-04-07,146,11.571429,4.928571,0,3.428571,65.742857,9.891429
2,2015-04-08,90,10.914286,4.571429,0,3.257143,63.542857,9.927143
3,2015-04-09,172,10.171429,0.571429,0,2.871429,59.614286,10.884286
4,2015-04-10,249,10.314286,0.071429,0,2.871429,53.485714,12.777143
...,...,...,...,...,...,...,...,...
1168,2020-10-27,535,11.942857,0.000000,0,2.185714,63.242857,12.734286
1169,2020-10-28,884,11.900000,0.000000,0,2.271429,60.771429,14.241429
1170,2020-10-29,695,11.514286,0.000000,0,2.157143,57.100000,14.635714
1171,2020-10-30,543,11.985714,0.000000,0,1.942857,55.871429,14.294286


In [4]:
pd.to_datetime(mosquitoDF['date'])

0      2015-04-06
1      2015-04-07
2      2015-04-08
3      2015-04-09
4      2015-04-10
          ...    
1168   2020-10-27
1169   2020-10-28
1170   2020-10-29
1171   2020-10-30
1172   2020-10-31
Name: date, Length: 1173, dtype: datetime64[ns]

In [5]:
mosquitoDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1173 non-null   object 
 1   mosquito      1173 non-null   int64  
 2   temp          1173 non-null   float64
 3   rain_per_day  1173 non-null   float64
 4   accum_rain    1173 non-null   int64  
 5   wind          1173 non-null   float64
 6   humidity      1173 non-null   float64
 7   sunshine      1173 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 73.4+ KB


In [6]:
mosquitoDF['year'] = mosquitoDF['date'].apply(lambda x : x.year)
mosquitoDF['month'] = mosquitoDF['date'].apply(lambda x : x.month)
mosquitoDF['day'] = mosquitoDF['date'].apply(lambda x : x.day)

AttributeError: ignored

In [ ]:
dataset = mosquitoDF.loc[:,'mosquito' :].astype('float')
dataset.isnull().sum()

In [ ]:
train = mosquitoDF.iloc[:round(len(mosquitoDF)*0.7)]
test = mosquitoDF.iloc[round(len(mosquitoDF)*0.7):]

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data=mosquitoDF.corr(method='spearman'), annot=True,fmt='.2f', linewidths=.5, cmap='Blues')

In [ ]:
dataset.columns

In [ ]:
sns.pairplot(dataset, x_vars=['temp', 'rain_per_day', 'accum_rain', 'wind',
       'humidity', 'sunshine', 'year', 'month', 'day'],y_vars=['mosquito'])

In [ ]:
dataset.head()

In [ ]:
X_train = dataset.iloc[:round(len(dataset)*0.85),1:-1]
y_train = dataset.iloc[:round(len(dataset)*0.85),0]
X_test = dataset.iloc[round(len(dataset)*0.85):,1:-1]
y_test = dataset.iloc[round(len(dataset)*0.85):,0]

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
def svr_gbr(name = '', train_x = '', train_y = '', test_x = '', test_y = '', SVR_C = 1000.0, gbr_n = 10) :

  svr = make_pipeline(StandardScaler(), SVR(C=SVR_C, epsilon=0.2))
  svr.fit(train_x, train_y)
  print(svr)
  print(f"SVM 훈련 세트 점수: {svr.score(train_x, train_y)} \nSVM 테스트 세트 점수 : {svr.score(test_x, test_y)}")

  gbr = GradientBoostingRegressor(n_estimators= gbr_n)
  gbr.fit(train_x, train_y)
  print(gbr)
  print(f"GBR 훈련 세트 점수: {gbr.score(train_x, train_y)} \nGBR 테스트 세트 점수 : {gbr.score(test_x, test_y)}")

  prediction_svm = svr.predict(test_x)
  prediction_gbr = gbr.predict(test_x)

  df = pd.DataFrame({'result' : test_y, name + 'SVM' : prediction_svm, name + 'gbr' : prediction_gbr})
  
  return df

In [ ]:
df = svr_gbr('test', X_train, y_train, X_test, y_test)
df.head()

In [ ]:
df['time'] = mosquitoDF['date']
df = df.set_index('time', drop = True)
df.head()

In [ ]:
df.tail()

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
plt.rcParams['figure.dpi'] = 100

In [ ]:
plt.plot(df.result,'r', label = 'actual')
plt.plot(df.testSVM, 'b', label = 'svm')
plt.plot(df.testgbr, 'g', label = 'gbr')
plt.legend()
plt.show()